In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
forest = RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [3]:
train = pd.read_csv('data/train_ver02.csv', index_col='STU_ID')
train.head()

,BYSTUWT,BYSEX,BYRACE,BYSTLANG,BYHOMLNG,BYSTLNG2,BYDOB_P,BYFCOMP,BYPARED,BYMOTHED,...,BYS24D,BYS24E,BYS28,BYS71A,BYS71B,BYS71C,BYS71D,BYS71E,BYS71F,F3ONET2CURR
STU_ID,,,,,,,,,,,,,,,,,,,,,
101102,28.2951,2.0,2.0,0.0,4.0,2,198605.0,1.0,5.0,5.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,29
101104,589.7248,2.0,7.0,1.0,1.0,4,198601.0,1.0,2.0,2.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,27
101105,235.7822,2.0,3.0,1.0,1.0,4,198607.0,5.0,2.0,2.0,...,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,39
101106,178.9513,2.0,4.0,0.0,2.0,2,198511.0,1.0,1.0,1.0,...,1.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,29
101108,192.4304,1.0,4.0,1.0,1.0,4,198607.0,1.0,6.0,6.0,...,2.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,53


In [4]:
y = train.F3ONET2CURR
x = train.drop(columns='F3ONET2CURR')

In [5]:
forest.fit(x,y)

RandomForestClassifier(n_estimators=300, n_jobs=-1)

In [6]:
imp = forest.feature_importances_
idx = np.argsort(imp)[::-1]
labels = x.columns
for i in range(len(labels)):
    print(labels[i], imp[idx[i]])

BYSTUWT 0.01742279333613537
BYSEX 0.016479379360383562
BYRACE 0.015097713094974293
BYSTLANG 0.014672343387955514
BYHOMLNG 0.014299604934637064
BYSTLNG2 0.014094465338077799
BYDOB_P 0.013969043935076876
BYFCOMP 0.013949857635874793
BYPARED 0.01392675676060892
BYMOTHED 0.013911756981077366
BYFATHED 0.013862092291882011
BYOCCUM 0.013786841479769643
BYOCCUF 0.013489323983745903
BYINCOME 0.013301251897365661
BYSES1QU 0.012977297374801775
BYSES2QU 0.012716716284842682
BYSCHPRG 0.012638905001958219
BYSQSTAT 0.012558921525924457
BYTXSTAT 0.012506386094126716
BYPQSTAT 0.012368784848988356
BYTXCSTD 0.01217704981694159
BYNELS2M 0.012065811941778842
BYNELS2R 0.011692072528453818
BYPISAME 0.011608644731146547
BYPISARE 0.011382333919033434
BYTXMIRR 0.011287190864255154
BYTXMSTD 0.011182242980015095
BYTX1MPP 0.010847599249367642
BYTX2MPP 0.010449331130180874
BYTX3MPP 0.010402797981031494
BYTX4MPP 0.009980670903507117
BYTX5MPP 0.009697432800567249
BYTXRIRR 0.009669530772353102
BYTXRSTD 0.0095392665958

In [7]:
x_filter = x[labels[imp>0.01]]
x_filter.shape

(9955, 30)

In [8]:
x.shape

(9955, 151)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [12]:
pipe = Pipeline([('전처리',StandardScaler()), ('알고리즘', KNeighborsClassifier())])

In [13]:
param = [
    {'알고리즘':[KNeighborsClassifier()],
     '전처리' : [StandardScaler()],
     '알고리즘__n_neighbors': np.arange(1,20),
     '알고리즘__weights': ['uniform','distance'],
     '알고리즘__p':[1,2]},
    {'알고리즘':[RandomForestClassifier()],
     '전처리': [MinMaxScaler(), None],
     '알고리즘__n_estimators': [300],
     '알고리즘__criterion': ['gini','entropy'],
     '알고리즘__max_depth': [None]+list(np.arange(5,20))}]

In [14]:
from sklearn.model_selection import GridSearchCV
grid_full = GridSearchCV(estimator=pipe, param_grid=param,
                         scoring='accuracy', cv=4, n_jobs=-1, verbose=1)
grid_small = GridSearchCV(estimator=pipe, param_grid=param,
                          scoring='accuracy', cv=4, n_jobs=-1, verbose=1)

In [15]:
grid_full.fit(x,y)

Fitting 4 folds for each of 140 candidates, totalling 560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 560 out of 560 | elapsed:  9.4min finished


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('전처리', StandardScaler()),
                                       ('알고리즘', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid=[{'알고리즘': [KNeighborsClassifier()],
                          '알고리즘__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                          '알고리즘__p': [1, 2],
                          '알고리즘__weights': ['uniform', 'distance'],
                          '전처리': [StandardScaler()]},
                         {'알고리즘': [RandomForestClassifier(max_depth=16,
                                                          n_estimators=300)],
                          '알고리즘__criterion': ['gini', 'entropy'],
                          '알고리즘__max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12,
                                              13, 14, 15, 16, 17, 18, 19],
                          '알고리즘__n_estimators': [300],
                         

In [16]:
grid_small.fit(x_filter, y)

Fitting 4 folds for each of 140 candidates, totalling 560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 560 out of 560 | elapsed:  5.8min finished


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('전처리', StandardScaler()),
                                       ('알고리즘', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid=[{'알고리즘': [KNeighborsClassifier()],
                          '알고리즘__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                          '알고리즘__p': [1, 2],
                          '알고리즘__weights': ['uniform', 'distance'],
                          '전처리': [StandardScaler()]},
                         {'알고리즘': [RandomForestClassifier(max_depth=12,
                                                          n_estimators=300)],
                          '알고리즘__criterion': ['gini', 'entropy'],
                          '알고리즘__max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12,
                                              13, 14, 15, 16, 17, 18, 19],
                          '알고리즘__n_estimators': [300],
                         

In [17]:
grid_full.best_score_

0.2522362462358493

In [18]:
grid_small.best_score_

0.23656560732173332

In [21]:
grid_full.best_estimator_

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [23]:
grid_small.best_estimator_

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=12, n_estimators=300))])